In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import time

In [2]:
# 如果是linux系统
trainPath = os.getcwd()+'/train.csv'
testPath = os.getcwd()+'/test.csv'
train_df = pd.read_csv(trainPath, index_col=0)
test_df = pd.read_csv(testPath, index_col=0)

/Users/xhxt2008/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# 如果是win系统

#train_df = pd.read_csv('train.csv', index_col=0)
#test_df = pd.read_csv('test.csv', index_col=0)

In [4]:
#去除y值
y_train = train_df.pop('acc_now_delinq')

In [5]:
#合并训练、测试集
all_df = pd.concat((train_df, test_df), axis = 0)

In [6]:
data = all_df

In [7]:
all_df.columns.values

array(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length',
       'home_ownership', 'annual_inc', 'verification_status', 'issue_d',
       'loan_status', 'pymnt_plan', 'desc', 'purpose', 'title', 'zip_code',
       'addr_state', 'dti', 'earliest_cr_line', 'mths_since_last_record',
       'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint',
       'verification_status_joint', 'tot_coll_amt', 'tot_cur_bal',
       'open_acc_6m', 'open_il_6m', 'open_il_12m', 'open_il_24m',
       'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m',
       'open_rv_24m'

In [8]:
#sub_grade

sub_grade_mapping = {'A1':0,'A2':1,'A3':2,'A4':3,'A5':4,
                     'B1':5,'B2':6,'B3':7,'B4':8,'B5':9,
                     'C1':10,'C2':11,'C3':12,'C4':13,'C5':14,
                     'D1':15,'D2':16,'D3':17,'D4':18,'D5':19,
                     'E1':20,'E2':21,'E3':22,'E4':23,'E5':24,
                     'F1':25,'F2':26,'F3':27,'F4':28,'F5':29,
                     'G1':30,'G2':31,'G3':32,'G4':33,'G5':34}

data.sub_grade = data.sub_grade.map(sub_grade_mapping)



In [9]:
'''
for delete: grade,zip_code,inq_last_12m,total_cu_tl,open_acc_6m,open_il_12m,open_il_24m,open_il_6m,total_bal_il,
open_rv_12m,open_rv_24m,max_bal_bc,all_util,inq_fi,policy_code,emp_title,emp_title,desc,title,zip_code,dti_joint,
annual_inc_joint,varification_status_joint,il_uti,mths_since_last_record,mths_since_last_major_derog
'''
feature_delete = {'grade','zip_code','inq_last_12m','total_cu_tl','open_acc_6m','open_il_12m','open_il_24m',
                  'open_il_6m','total_bal_il','open_rv_24m','open_rv_12m','max_bal_bc','all_util','inq_fi',
                 'emp_title','policy_code','desc','title','dti_joint','annual_inc_joint',
                 'varification_status_joint','il_uti','mths_since_last_record','mths_since_last_major_derog'}
data = data.drop(feature_delete,axis=1)

In [10]:
#for binarilization：pymnt_plan,term
feature_for_binary = {'term','pymnt_plan'}
def binary(feature_for_binary,data):
    for items in feature_for_binary: 
        mapping = {label:idx for idx,label in enumerate(set(data[items]))}  
        data[items] = data[items].map(mapping) 
    return data

In [11]:
binary(feature_for_binary,data)
temp = data.copy()

In [12]:
#emp_length
emp_length_mapping = {'10+ years':10,'9 years':9,'8 years':8,'7 years':7,'6 years':6,
                       '5 years':5,'4 years':4,'3 years':3,'2 years':2,'1 year':1,'< 1 year':0}
data.emp_length = data.emp_length.map(emp_length_mapping) 

In [13]:
# 少量null值的处理，用均值替代：emp_length, revol_util, annual_inc, total_acc, earliest_cr_line
feature_fill_null = {'emp_length','revol_util','annual_inc','total_acc','earliest_cr_line'}
def fill_null(feature_fill_null,data):
    for items in feature_fill_null:
        try:
            time.strptime(data[items].mode()[0],"%b-%Y")
            data[items] = data[items].fillna(data[items].mode()[0])
            print data[items].mode()[0]
            print ('yes')
        except:
            data[items] = data[items].fillna(data[items].mean())
            print data[items].mean()
            print('no')
#         if time.strptime(feature_dropna.mode()[0],"%b-%Y"):
#             data[items] = data[items].fillna(feature_dropna.mode())
#             print feature_dropna.mode()
#         else:
#             data[items] = data[items].fillna(feature_dropna.mean())
    return data

In [14]:
fill_null(feature_fill_null,data)

25.2680261453
no
Aug-2001
yes
55.0676932314
no
75027.5877608
no
6.00869499165
no


,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,sub_grade,emp_title,emp_length,home_ownership,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
member_id,,,,,,,,,,,,,,,,,,,,,
13599771,8875.0,8875.0,8875.0,0,18.25,321.97,17,Inventory/Part time Sup,8.000000,MORTGAGE,...,NaN,NaN,NaN,NaN,NaN,NaN,16700.0,NaN,NaN,NaN
1491956,12500.0,12500.0,12475.0,0,8.90,396.92,4,Bel Brands USA,1.000000,MORTGAGE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64687987,33600.0,33600.0,33600.0,1,18.55,863.31,21,Assistant Principal,9.000000,RENT,...,NaN,NaN,NaN,NaN,NaN,NaN,41300.0,NaN,NaN,NaN
8446850,17000.0,17000.0,17000.0,1,9.71,358.78,5,Cook Childrens Medical Center,10.000000,MORTGAGE,...,NaN,NaN,NaN,NaN,NaN,NaN,22800.0,NaN,NaN,NaN
58471388,14000.0,14000.0,14000.0,1,16.99,347.87,17,Cook,5.000000,RENT,...,NaN,NaN,NaN,NaN,NaN,NaN,10300.0,NaN,NaN,NaN
5337525,24000.0,24000.0,24000.0,1,17.27,599.96,14,walmart distribution cntr,10.000000,OWN,...,NaN,NaN,NaN,NaN,NaN,NaN,24600.0,NaN,NaN,NaN
30165519,12000.0,12000.0,12000.0,1,16.29,293.67,16,Officer,1.000000,MORTGAGE,...,NaN,NaN,NaN,NaN,NaN,NaN,8900.0,NaN,NaN,NaN
1909391,23325.0,23325.0,23325.0,1,20.49,624.35,20,stallion production services,4.000000,MORTGAGE,...,NaN,NaN,NaN,NaN,NaN,NaN,14800.0,NaN,NaN,NaN
43182363,15000.0,15000.0,15000.0,1,10.49,322.34,7,Sr. Electrical Designer,3.000000,MORTGAGE,...,NaN,NaN,NaN,NaN,NaN,NaN,36000.0,NaN,NaN,NaN


In [16]:
temp.earliest_cr_line.isnull().sum()

29

In [28]:
temp1 = data.copy()

In [32]:
#for get_dummies: home_ownership, verification_status, loan_status,purpose

feature_for_dummy = {'home_ownership','verification_status','loan_status','purpose'}
def multi_get_dummies(feature_for_dummy,data):
    for items in feature_for_dummy: 
        data = data.join(pd.get_dummies(data[items],prefix=items))
        data.pop(items)
    return data    

In [35]:
loan_status_mapping = {'Charged Off':'CF','Fully Paid':'FP','Current':'Current','In Grace Period':'IGP','Late (31-120 days)':'L31-120',
                       'Issued':'Issued','Does not meet the credit policy. Status:Charged Off':'NotMeetCF','Does not meet the credit policy. Status:Fully Paid':'NotMeetFP',
                       'Default':'Default','Late (16-30 days)':'L16-30'}
data.loan_status = data.loan_status.map(loan_status_mapping) 

In [36]:
data.loan_status.unique()

array(['CF', 'FP', 'Current', 'IGP', 'L31-120', 'Issued', 'NotMeetCF',
       'NotMeetFP', 'Default', 'L16-30'], dtype=object)

In [37]:
data = multi_get_dummies(feature_for_dummy,data)

In [43]:
data.columns.values

array(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'sub_grade', 'emp_title', 'emp_length',
       'home_ownership', 'annual_inc', 'verification_status', 'issue_d',
       'loan_status', 'pymnt_plan', 'desc', 'purpose', 'title', 'zip_code',
       'addr_state', 'dti', 'earliest_cr_line', 'mths_since_last_record',
       'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint',
       'verification_status_joint', 'tot_coll_amt', 'tot_cur_bal',
       'open_acc_6m', 'open_il_6m', 'open_il_12m', 'open_il_24m',
       'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m',
       'open_rv_24m', 'max_ba

In [27]:

pd.Period(data.earliest_cr_line.mode()[0])

Period('2001-08', 'M')

In [30]:
all_df[all_df.earliest_cr_line.isnull()].loc[:,['earliest_cr_line' ,'total_acc','pub_rec']]

,earliest_cr_line,total_acc,pub_rec
member_id,,,
70473,NaN,NaN,NaN
83132,NaN,NaN,NaN
70651,NaN,NaN,NaN
72992,NaN,NaN,NaN
74469,NaN,NaN,NaN
79912,NaN,NaN,NaN
73890,NaN,NaN,NaN
79878,NaN,NaN,NaN
72816,NaN,NaN,NaN


In [36]:
data3 = all_df[all_df.inq_last_12m.notnull()].loc[:,['inq_last_12m' ,'total_cu_tl','open_acc_6m','open_il_12m','open_il_24m','open_il_6m','total_bal_il','open_rv_12m','open_rv_24m','max_bal_bc','all_util','inq_fi','all_util']]

In [41]:
data3 = data3.join(y_train)

In [47]:
data3.acc_now_delinq.sum()

78.0

In [49]:
y_train.sum()

3293L

In [53]:
y_train.astype('string').describe()

count     709903
unique         2
top            0
freq      706610
Name: acc_now_delinq, dtype: object

In [90]:
data4 = all_df.loc[all_df.mths_since_last_record.notnull()]

In [91]:
data4

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
member_id,,,,,,,,,,,,,,,,,,,,,
13599771,8875.0,8875.0,8875.00000,36 months,18.25,321.97,D,17,Inventory/Part time Sup,8 years,...,NaN,NaN,NaN,NaN,NaN,NaN,16700.0,NaN,NaN,NaN
30165519,12000.0,12000.0,12000.00000,60 months,16.29,293.67,D,16,Officer,1 year,...,NaN,NaN,NaN,NaN,NaN,NaN,8900.0,NaN,NaN,NaN
67687466,6000.0,6000.0,6000.00000,36 months,6.89,184.97,A,2,Office Manager/Bookkeeper,3 years,...,NaN,NaN,NaN,NaN,NaN,NaN,25700.0,NaN,NaN,NaN
11918997,13300.0,13300.0,13300.00000,36 months,17.57,477.97,D,16,Business mgr,7 years,...,NaN,NaN,NaN,NaN,NaN,NaN,20100.0,NaN,NaN,NaN
42586279,6000.0,6000.0,6000.00000,36 months,21.99,229.12,E,24,Teacher,10+ years,...,NaN,NaN,NaN,NaN,NaN,NaN,9200.0,NaN,NaN,NaN
47954850,19850.0,19850.0,19850.00000,36 months,14.65,684.71,C,14,Operations Supervisor,10+ years,...,NaN,NaN,NaN,NaN,NaN,NaN,32100.0,NaN,NaN,NaN
8549779,14000.0,14000.0,14000.00000,60 months,17.56,352.17,D,15,Hoosier Crane Service,8 years,...,NaN,NaN,NaN,NaN,NaN,NaN,13200.0,NaN,NaN,NaN
63456897,5000.0,5000.0,5000.00000,36 months,8.18,157.10,B,5,NaN,n/a,...,NaN,NaN,NaN,NaN,NaN,NaN,13000.0,NaN,NaN,NaN
63058421,12800.0,12800.0,12800.00000,60 months,12.69,289.22,C,11,Police Officer,4 years,...,NaN,NaN,NaN,NaN,NaN,NaN,23900.0,NaN,NaN,NaN


In [92]:
data4 = data4.join(y_train)

In [93]:
data4.loc[:,['mths_since_last_record','acc_now_delinq']]

,mths_since_last_record,acc_now_delinq
member_id,,
13599771,50.0,0.0
30165519,114.0,0.0
67687466,93.0,0.0
11918997,105.0,0.0
42586279,65.0,0.0
47954850,115.0,0.0
8549779,88.0,0.0
63456897,51.0,0.0
63058421,80.0,0.0


In [94]:
data4.acc_now_delinq.sum()

472.0